In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm # 폰트연결 시켜주는 모듈
font_name = fm.FontProperties(fname = "C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family = font_name)

# 마이너스 사인 해결
import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Flatten, Dense, Input, concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import SGD, Adam
import matplotlib .pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score

from datetime import datetime
import cv2
import zipfile
import shutil
import os
import glob

from tensorflow.keras.applications import Xception, ResNet50
from tensorflow.keras.applications import InceptionV3, MobileNet

In [4]:
import mglearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.datasets import load_iris, fetch_california_housing, load_breast_cancer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report, roc_curve
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC, SVR
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.metrics import mean_squared_error

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.cluster import KMeans

from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

In [37]:
import chardet
tom2 = "C:\\YOONJINSEOK\\Python work\\AI\\project\\smart_tomato\\01.데이터\\1.Training\\원천데이터\\TS_Timeseries\\tom2"

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

tom2_file_list = glob.glob(os.path.join(tom2, "양액기 수집정보", "*"))

lp = {}

for file_path in tom2_file_list:
    encoding = detect_encoding(file_path)

    with open(file_path, 'r', encoding=encoding) as f:
        columns = f.readline().strip().split(';')
        columns = [col.replace('"', '') for col in columns]
        columns = [col.strip() for col in columns]

    
        df = pd.read_csv(f, sep=';', names=columns, header=None)

    df = df.replace({'"': ''}, regex=True)


    file_name = os.path.splitext(os.path.basename(file_path))[0]
    lp[file_name] = df
    
for key in lp.keys():
    lp[key]["Time"] = pd.to_datetime(lp[key]["Time"])
    lp[key]["Time"] = lp[key]["Time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    
keys = sorted(list(lp.keys()))

merged = lp[keys[0]]

for key in keys[1:]:
    merged = pd.merge(merged, lp[key], on='Time', how='inner')

print(merged.head(3))

                  Time  co2       습도       광량     내부온도     상부온도  급액 EC  배액 EC  \
0  2021-11-24 10:32:00    0  96.1895  91.8367  16.0042  15.9369   2.91   4.95   
1  2021-11-24 10:33:00  284  96.6069  83.4555  16.0042  15.9030   2.91   4.95   
2  2021-11-24 10:34:00  285  95.7769  65.8779  16.1056  15.9369   2.91   4.95   

   pH_Supply  pH_Drain  Temp_Soil  Temp_Supply  Temp_Drain  Scale_Supply  \
0       4.89      6.29      17.56          NaN         NaN         16.18   
1       4.89      6.29      17.56          NaN         NaN         16.18   
2       4.90      6.28      17.62          NaN         NaN         16.18   

   Scale_Drain  Deviation(Diff)        BRS  Real leaves  
0         0.16             0.31  19.930000    20.240000  
1         0.16             0.92  20.270000    21.190001  
2         0.16             0.98  20.690001    21.670000  


In [38]:
merged

,Time,co2,습도,광량,내부온도,상부온도,급액 EC,배액 EC,pH_Supply,pH_Drain,Temp_Soil,Temp_Supply,Temp_Drain,Scale_Supply,Scale_Drain,Deviation(Diff),BRS,Real leaves
0,2021-11-24 10:32:00,0,96.1895,91.8367,16.0042,15.9369,2.91,4.95,4.89,6.29,17.56,NaN,NaN,16.18,0.16,0.31,19.930000,20.240000
1,2021-11-24 10:33:00,284,96.6069,83.4555,16.0042,15.9030,2.91,4.95,4.89,6.29,17.56,NaN,NaN,16.18,0.16,0.92,20.270000,21.190001
2,2021-11-24 10:34:00,285,95.7769,65.8779,16.1056,15.9369,2.91,4.95,4.90,6.28,17.62,NaN,NaN,16.18,0.16,0.98,20.690001,21.670000
3,2021-11-24 10:35:00,282,96.1914,63.5342,16.0718,15.9030,2.92,4.96,4.90,6.28,17.62,NaN,NaN,16.18,0.16,0.77,20.670000,21.440001
4,2021-11-24 10:36:00,282,95.9838,67.1261,16.1056,15.9030,2.92,4.96,4.90,6.29,17.62,NaN,NaN,16.18,0.16,0.58,20.860001,21.440001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79458,2022-01-31 23:55:00,391,100.0000,0.0000,10.6108,10.1057,2.38,3.59,5.67,6.18,17.38,NaN,NaN,17.29,0.63,0.17,12.910000,12.740000
79459,2022-01-31 23:56:00,395,100.0000,0.0000,10.5772,10.0383,2.38,3.59,5.67,6.18,17.38,NaN,NaN,17.29,0.63,0.19,12.910000,12.720000
79460,2022-01-31 23:57:00,393,100.0000,0.0000,10.5772,10.0720,2.38,3.59,5.67,6.18,17.38,NaN,NaN,17.29,0.64,0.16,12.980000,12.820000
79461,2022-01-31 23:58:00,395,100.0000,0.0000,10.5772,10.0720,2.38,3.59,5.66,6.18,17.38,NaN,NaN,17.29,0.63,0.10,13.110000,13.010000


In [7]:
print(lp["MC_temp"][["Temp_Supply","Temp_Drain"]].isnull().sum())
print(len(lp["MC_temp"][["Temp_Supply","Temp_Drain"]]))

Temp_Supply    96585
Temp_Drain     96585
dtype: int64
139598


## 환경제어기 양액기 데이터(1번 데이터)

In [8]:
file_path = "smart_tomato/01.데이터/1.Training/원천데이터/TS_Timeseries"
file_list = []
env_list = []
dfs = []
to_file = os.path.join(file_path, "tom2")
#print(to_file)


to_file_env = os.path.join(to_file, "00_(환경제어기, 양액기)")
env_list.append(to_file_env)
#print(env_list)

for file in env_list:
    file_name = glob.glob(file+"/*.xlsx")
    
    for names in file_name:
        file_list.append(names)
#print(file_list)
        
for i in range(len(file_list)):
    
    try:
        df = pd.read_excel(file_list[i])
        
    except UnicodeDecodeError as e :
        
        print(file_list[i])
        
    dfs.append(df)

In [9]:
len(dfs)

18

In [10]:
column_list = []
for i in range(18):
    column_list.append(dfs[i].columns)

col_df = pd.DataFrame(column_list)
col_df
# 컬럼명과 숫자는 다 맞네

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,날짜(관수관리데이터),WEEK(관수관리데이터),공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도
1,날짜(관수관리데이터),WEEK(관수관리데이터),공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도
2,날짜(관수관리데이터),WEEK(관수관리데이터),공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도
3,날짜(관수관리데이터),WEEK(관수관리데이터),공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도
4,날짜(관수관리데이터),WEEK(관수관리데이터),공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도
5,날짜(관수관리데이터),WEEK(관수관리데이터),공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도
6,날짜(관수관리데이터),WEEK(관수관리데이터),공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도
7,날짜(관수관리데이터),WEEK(관수관리데이터),공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도
8,날짜(관수관리데이터),WEEK(관수관리데이터),공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도
9,날짜(관수관리데이터),WEEK(관수관리데이터),공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도


In [11]:
con_df1 = pd.concat([dfs[0], dfs[1]], axis = 0)
con_df2 = pd.concat([con_df1, dfs[2]], axis = 0)
con_df3 = pd.concat([con_df2, dfs[3]], axis = 0)
con_df4 = pd.concat([con_df3, dfs[4]], axis = 0)
con_df5 = pd.concat([con_df4, dfs[5]], axis = 0)
con_df6 = pd.concat([con_df5, dfs[6]], axis = 0)
con_df7 = pd.concat([con_df6, dfs[7]], axis = 0)
con_df8 = pd.concat([con_df7, dfs[8]], axis = 0)
con_df9 = pd.concat([con_df8, dfs[9]], axis = 0)
con_df10 = pd.concat([con_df9, dfs[10]], axis = 0)
con_df11 = pd.concat([con_df10, dfs[11]], axis = 0)
con_df12 = pd.concat([con_df11, dfs[12]], axis = 0)
con_df13 = pd.concat([con_df12, dfs[13]], axis = 0)
con_df14 = pd.concat([con_df13, dfs[14]], axis = 0)
con_df15 = pd.concat([con_df14, dfs[15]], axis = 0)
con_df16 = pd.concat([con_df15, dfs[16]], axis = 0)
con_df17 = pd.concat([con_df16, dfs[17]], axis = 0)

In [12]:
con_df17.info()
# 애초에 날짜가 1일 단위라서 데이터량을 늘릴 방법이 애매함

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 0 to 10
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   날짜(관수관리데이터)    188 non-null    datetime64[ns]
 1   WEEK(관수관리데이터)  187 non-null    float64       
 2   공급ec           187 non-null    float64       
 3   공급ph           187 non-null    float64       
 4   공급량            187 non-null    float64       
 5   배액ec           187 non-null    float64       
 6   배액ph           187 non-null    float64       
 7   배액량            187 non-null    float64       
 8   공급횟수           187 non-null    float64       
 9   1회공급량          179 non-null    float64       
 10  배액율            187 non-null    float64       
 11  함수율편차          187 non-null    float64       
 12  배지온도           0 non-null      float64       
dtypes: datetime64[ns](1), float64(12)
memory usage: 20.6 KB


In [20]:
df1 = con_df17
df1

,날짜(관수관리데이터),WEEK(관수관리데이터),공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도
0,2021-10-11,41.0,2.80,6.20,310.0,0.00,0.00,0.0,3.0,103.333333,0.000000,0.0,NaN
1,2021-10-12,41.0,2.80,5.50,170.0,0.00,0.00,0.0,2.0,85.000000,0.000000,0.0,NaN
2,2021-10-13,41.0,2.30,6.35,568.0,5.10,6.30,1405.0,11.0,51.636364,0.412265,0.0,NaN
3,2021-10-14,41.0,2.85,6.35,1325.0,5.05,6.30,3475.0,14.0,94.642857,0.437107,0.0,NaN
4,2021-10-15,41.0,2.80,6.30,710.0,5.15,6.20,770.0,7.0,101.428571,0.180751,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,2021-09-30,39.0,2.80,6.40,2090.0,4.85,6.65,5510.0,22.0,95.000000,0.439394,0.0,NaN
7,2021-10-01,39.0,2.65,6.10,2025.0,4.45,6.25,4300.0,21.0,96.428571,0.353909,0.0,NaN
8,2021-10-02,39.0,2.50,6.50,1790.0,4.10,6.35,3785.0,19.0,94.210526,0.352421,0.0,NaN
9,2021-10-03,39.0,2.50,6.30,1930.0,4.00,6.30,3475.0,20.0,96.500000,0.300086,0.0,NaN


In [21]:
cols = ['날짜', '주차', '공급ec', '공급ph', '공급량', '배액ec', '배액ph',
       '배액량', '공급횟수', '1회공급량', '배액율', '함수율편차', '배지온도']
df1.columns = cols

In [22]:
df1

,날짜,주차,공급ec,공급ph,공급량,배액ec,배액ph,배액량,공급횟수,1회공급량,배액율,함수율편차,배지온도
0,2021-10-11,41.0,2.80,6.20,310.0,0.00,0.00,0.0,3.0,103.333333,0.000000,0.0,NaN
1,2021-10-12,41.0,2.80,5.50,170.0,0.00,0.00,0.0,2.0,85.000000,0.000000,0.0,NaN
2,2021-10-13,41.0,2.30,6.35,568.0,5.10,6.30,1405.0,11.0,51.636364,0.412265,0.0,NaN
3,2021-10-14,41.0,2.85,6.35,1325.0,5.05,6.30,3475.0,14.0,94.642857,0.437107,0.0,NaN
4,2021-10-15,41.0,2.80,6.30,710.0,5.15,6.20,770.0,7.0,101.428571,0.180751,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,2021-09-30,39.0,2.80,6.40,2090.0,4.85,6.65,5510.0,22.0,95.000000,0.439394,0.0,NaN
7,2021-10-01,39.0,2.65,6.10,2025.0,4.45,6.25,4300.0,21.0,96.428571,0.353909,0.0,NaN
8,2021-10-02,39.0,2.50,6.50,1790.0,4.10,6.35,3785.0,19.0,94.210526,0.352421,0.0,NaN
9,2021-10-03,39.0,2.50,6.30,1930.0,4.00,6.30,3475.0,20.0,96.500000,0.300086,0.0,NaN


In [33]:
df1.columns

Index(['날짜', '주차', '공급ec', '공급ph', '공급량', '배액ec', '배액ph', '배액량', '공급횟수',
       '1회공급량', '배액율', '함수율편차', '배지온도'],
      dtype='object')

In [14]:
con_df17.isnull().sum()
# 배지온도 전체 null

날짜(관수관리데이터)        0
WEEK(관수관리데이터)      1
공급ec               1
공급ph               1
공급량                1
배액ec               1
배액ph               1
배액량                1
공급횟수               1
1회공급량              9
배액율                1
함수율편차              1
배지온도             188
dtype: int64

In [39]:
cols = ['날짜', 'CO2', '습도', '광량', '내부온도', '상부온도', '급액 EC', '배액 EC',
       '급액 pH', '배액 pH', '토양온도', 'Temp_Supply', 'Temp_Drain',
       'Scale_Supply', 'Scale_Drain', 'Deviation(Diff)', 'BRS', '생존엽수']
merged.columns = cols


In [40]:
df2 = merged

In [41]:
arr_list = []
for i in df2["날짜"].values:
    arr_list.append(i.split(" ")[0])
np.array(arr_list)

array(['2021-11-24', '2021-11-24', '2021-11-24', ..., '2022-01-31',
       '2022-01-31', '2022-01-31'], dtype='<U10')

In [42]:
df2["날짜"] = np.array(arr_list)

In [44]:
df2

,날짜,CO2,습도,광량,내부온도,상부온도,급액 EC,배액 EC,급액 pH,배액 pH,토양온도,Temp_Supply,Temp_Drain,Scale_Supply,Scale_Drain,Deviation(Diff),BRS,생존엽수
0,2021-11-24,0,96.1895,91.8367,16.0042,15.9369,2.91,4.95,4.89,6.29,17.56,NaN,NaN,16.18,0.16,0.31,19.930000,20.240000
1,2021-11-24,284,96.6069,83.4555,16.0042,15.9030,2.91,4.95,4.89,6.29,17.56,NaN,NaN,16.18,0.16,0.92,20.270000,21.190001
2,2021-11-24,285,95.7769,65.8779,16.1056,15.9369,2.91,4.95,4.90,6.28,17.62,NaN,NaN,16.18,0.16,0.98,20.690001,21.670000
3,2021-11-24,282,96.1914,63.5342,16.0718,15.9030,2.92,4.96,4.90,6.28,17.62,NaN,NaN,16.18,0.16,0.77,20.670000,21.440001
4,2021-11-24,282,95.9838,67.1261,16.1056,15.9030,2.92,4.96,4.90,6.29,17.62,NaN,NaN,16.18,0.16,0.58,20.860001,21.440001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79458,2022-01-31,391,100.0000,0.0000,10.6108,10.1057,2.38,3.59,5.67,6.18,17.38,NaN,NaN,17.29,0.63,0.17,12.910000,12.740000
79459,2022-01-31,395,100.0000,0.0000,10.5772,10.0383,2.38,3.59,5.67,6.18,17.38,NaN,NaN,17.29,0.63,0.19,12.910000,12.720000
79460,2022-01-31,393,100.0000,0.0000,10.5772,10.0720,2.38,3.59,5.67,6.18,17.38,NaN,NaN,17.29,0.64,0.16,12.980000,12.820000
79461,2022-01-31,395,100.0000,0.0000,10.5772,10.0720,2.38,3.59,5.66,6.18,17.38,NaN,NaN,17.29,0.63,0.10,13.110000,13.010000


In [81]:
## 광량 하루치 sum 값 구하기
df2["광량"].groupby(df2.날짜).sum()
# 값이 별로 안큰데?

날짜
2021-11-24    44.513921
2021-11-25     8.617343
2021-11-26    47.869430
2021-11-27     0.008158
2021-11-28    64.037173
                ...    
2022-01-27    43.079355
2022-01-28    44.944171
2022-01-29    43.769418
2022-01-30    61.215831
2022-01-31    44.059294
Name: 광량, Length: 63, dtype: float64

In [46]:
df2 = df2.groupby("날짜").mean()

In [48]:
df2 = df2.reset_index()

In [56]:
print(df2.info(), df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   날짜               63 non-null     datetime64[ns]
 1   CO2              63 non-null     float64       
 2   습도               63 non-null     float64       
 3   광량               63 non-null     float64       
 4   내부온도             63 non-null     float64       
 5   상부온도             63 non-null     float64       
 6   급액 EC            63 non-null     float64       
 7   배액 EC            63 non-null     float64       
 8   급액 pH            63 non-null     float64       
 9   배액 pH            63 non-null     float64       
 10  토양온도             63 non-null     float64       
 11  Temp_Supply      0 non-null      float64       
 12  Temp_Drain       0 non-null      float64       
 13  Scale_Supply     63 non-null     float64       
 14  Scale_Drain      63 non-null     float64    

In [55]:
df2["날짜"] = pd.to_datetime(df2["날짜"])

In [58]:
df3 = pd.merge(df1, df2, how = "outer")

In [64]:
df3["함수율편차"].value_counts()
#뺴자

0.0    187
Name: 함수율편차, dtype: int64

In [65]:
df3["광량"].value_counts()
# 하루치를 따로 sum해야 할듯

35.854265    4
32.395856    4
30.240068    3
32.565586    3
65.841567    3
            ..
4.015431     1
28.875016    1
20.814448    1
0.020673     1
44.059294    1
Name: 광량, Length: 63, dtype: int64

In [74]:
df3["내부온도"].value_counts()

13.678082    4
14.020756    4
13.703934    3
12.652038    3
14.074620    3
            ..
13.236278    1
14.236340    1
13.599956    1
12.669242    1
13.858990    1
Name: 내부온도, Length: 63, dtype: int64

In [75]:
df3.columns

Index(['날짜', '주차', '공급ec', '공급ph', '공급량', '배액ec', '배액ph', '배액량', '공급횟수',
       '1회공급량', '배액율', '함수율편차', '배지온도', 'CO2', '습도', '광량', '내부온도', '상부온도',
       '급액 EC', '배액 EC', '급액 pH', '배액 pH', '토양온도', 'Temp_Supply', 'Temp_Drain',
       'Scale_Supply', 'Scale_Drain', 'Deviation(Diff)', 'BRS', '생존엽수'],
      dtype='object')

In [83]:
df4 = df3[['날짜', '1회공급량', '공급ec', '공급ph', 'CO2', '습도', '광량', '내부온도', '상부온도']]
df4

,날짜,1회공급량,공급ec,공급ph,CO2,습도,광량,내부온도,상부온도
0,2021-10-11,103.333333,2.80,6.20,NaN,NaN,NaN,NaN,NaN
1,2021-10-11,103.333333,2.80,6.20,NaN,NaN,NaN,NaN,NaN
2,2021-10-12,85.000000,2.80,5.50,NaN,NaN,NaN,NaN,NaN
3,2021-10-13,51.636364,2.30,6.35,NaN,NaN,NaN,NaN,NaN
4,2021-10-14,94.642857,2.85,6.35,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
183,2021-09-29,96.666667,2.80,6.50,NaN,NaN,NaN,NaN,NaN
184,2021-09-30,95.000000,2.80,6.40,NaN,NaN,NaN,NaN,NaN
185,2021-10-01,96.428571,2.65,6.10,NaN,NaN,NaN,NaN,NaN
186,2021-10-02,94.210526,2.50,6.50,NaN,NaN,NaN,NaN,NaN


In [85]:
df4.to_csv("farm2_df.csv", encoding = "utf-8")